# TP4 Alejandro balladares 101118

In [1]:
import pandas as pd
import numpy as np

In [2]:
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

In [3]:
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

In [4]:
!pip install category_encoders

In [5]:
import matplotlib.pyplot as plt
import seaborn as sns

from datetime import datetime
from sklearn import linear_model
from sklearn.metrics import mean_squared_error

import xgboost as xgb
from xgboost import XGBRegressor

import category_encoders as ce
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import TfidfTransformer

import warnings
warnings.filterwarnings('ignore')

from IPython.display import display
pd.options.display.max_columns = None


from numpy.random import seed
seed(1)
#from tensorflow import set_random_seed
#set_random_seed(2)

from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences

from keras.models import Sequential
from keras.layers import Dense, Embedding, LSTM, SpatialDropout1D
from keras.utils.np_utils import to_categorical
from keras.callbacks import EarlyStopping
from keras.layers import Dropout
import re
import nltk
from nltk import word_tokenize

nltk.download('stopwords')
#nltk.download('brown')

from nltk.corpus import stopwords
stops = set(stopwords.words('english'))
#punctuation = list(string.punctuation)
STOPWORDS = set(stopwords.words('english'))
#from bs4 import BeautifulSoup

/usr/local/lib/python3.7/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [6]:
#1PbJBh7uDOaWqZ5GOrEFS3EnGNS4OgB58

id_silkroad_items = '1PbJBh7uDOaWqZ5GOrEFS3EnGNS4OgB58'
downloaded = drive.CreateFile({'id': id_silkroad_items})
downloaded.GetContentFile('silkroad_items.csv')

df_silkroad_items = pd.read_csv('silkroad_items.csv')
#df_silkroad_items.head().T

# Tarea 3: Regresión sobre el precio del ítem

¿Se puede predecir el precio del ítem en base a su información? ¿Qué tan bien funciona en
validación? ¿Qué tan bien funciona años después en versus?


In [7]:
def agregarFechas(df, columna):
  df[columna] = pd.to_datetime(df[columna])
  df['Fecha'] = df[columna].transform(lambda x: x.date())

  df['Año'] = df['Fecha'].transform(lambda x: x.year)
  df['Mes'] = df['Fecha'].transform(lambda x: x.month)
  df['Dia'] = df['Fecha'].transform(lambda x: x.day)
  #df['Dia_de_la_semana'] = df['Fecha'].transform(lambda x: x.day_name())

#agregarFechas(df_silkroad_items, 'timestamp')

In [8]:
#df_silkroad_items_aux = df_silkroad_items.copy() #no borrarlo por las dudas
df_train = df_silkroad_items.copy()

# Armo mis sets

In [9]:
categoricas_binary = ['vendor','category','ships_from', 'ships_to']
categorica_texto = ['title', 'description']

limite = 10.00
valor_BTC = 1 #400
df_train['price'] = df_train['price'].transform(lambda x:( x*valor_BTC if x < limite else limite*valor_BTC))

In [10]:
limite = int(len(df_train)*0.7)

train = df_train[:limite]
test = df_train[limite:]

#-----------------------------

X_train_text = train['title']
Y_train = train['price']

Y_test = test['price']
X_test_text = test['title']


# Modelo

In [ ]:
def intento_final():
  MAX_NB_WORDS = 50000
  MAX_SEQUENCE_LENGTH = 30
  EMBEDDING_DIM = 100

  tokenizer = Tokenizer(num_words=MAX_NB_WORDS, filters='!"#$%&()*+,-./:;<=>?@[\]^_`{|}~', lower=True)
  tokenizer.fit_on_texts(df_train['title'].values)

  word_index = tokenizer.word_index
  print('Found %s unique tokens.' % len(word_index))

  X = tokenizer.texts_to_sequences(train['title'].values)
  X = pad_sequences(X, maxlen=MAX_SEQUENCE_LENGTH)
  Y = Y_train.values
  
  model = Sequential()
  model.add(Embedding(MAX_NB_WORDS, EMBEDDING_DIM, input_length=X.shape[1]))
  model.add(SpatialDropout1D(0.2))
  model.add(LSTM(100, dropout=0.2, recurrent_dropout=0.2))

  model.add(Dense(1, activation=None))
  model.compile(optimizer='adam', loss='mean_squared_error',metrics=['mae'])
    
  print(model.summary())

  epochs = 1
  batch_size = 64

  history = model.fit(X, Y, epochs=epochs, batch_size=batch_size,validation_split=0.1,callbacks=[EarlyStopping(monitor='val_loss', patience=3, min_delta=0.0001)])

  X_test2 = tokenizer.texts_to_sequences(test['title'].values)
  X_test2 = pad_sequences(X_test2, maxlen=MAX_SEQUENCE_LENGTH)

  accr = model.evaluate(X_test2,Y_test)
  print('Test set\n  Loss: {:0.3f}\n  Accuracy: {:0.3f}'.format(accr[0],accr[1]))
  return model, tokenizer

red, tokenizer = intento_final()

Found 13227 unique tokens.
Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 30, 100)           5000000   
                                                                 
 spatial_dropout1d (SpatialD  (None, 30, 100)          0         
 ropout1D)                                                       
                                                                 
 lstm (LSTM)                 (None, 100)               80400     
                                                                 
 dense (Dense)               (None, 1)                 101       
                                                                 
Total params: 5,080,501
Trainable params: 5,080,501
Non-trainable params: 0
_________________________________________________________________
None
2834/3949 [====================>.........] - ETA: 3:00 - loss: 623602.7500 - mae

# Test

In [ ]:
#archivo versus https://drive.google.com/file/d/1bKCo4VeH1owB8jGlwIT6NlljsbCSHs0X/view?usp=sharing

id_versus = '1bKCo4VeH1owB8jGlwIT6NlljsbCSHs0X'
downloaded = drive.CreateFile({'id': id_versus})
downloaded.GetContentFile('versus.csv')
df_test = pd.read_csv('versus.csv')

In [ ]:
df_test['price'] = df_test['price'].str.replace('EUR','')
df_test['price'] = df_test['price'].transform(lambda x: float(x))


In [ ]:
df_test.head()

In [ ]:
def test2(model, tokenizer, df_test):
  MAX_SEQUENCE_LENGTH = 30
  X = tokenizer.texts_to_sequences(df_test['title'].values)
  X = pad_sequences(X, maxlen=MAX_SEQUENCE_LENGTH)
  Y = df_test['price'].values

  preds = model.predict(X)  
  score = model.evaluate(X, Y, verbose=1)
  print('Test set\n  Loss: {:0.3f}\n  Accuracy: {:0.3f}'.format(score[0],score[1]))

test2(red, tokenizer, df_test)